In [1]:
import numpy as np
import pandas as pd
#import math
import sys,os

In [2]:
sys.path.append(os.path.abspath(os.path.join('../utils')))
from cleaner import *
from helper import *
from plot import *
from outlier_handler import OutlierHandler

In [3]:
df = pd.read_csv("../data/clean_data.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 48 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   Unnamed: 0                      146887 non-null  int64  
 1   Bearer Id                       146887 non-null  int64  
 2   Start                           146887 non-null  object 
 3   Start ms                        146887 non-null  float64
 4   End                             146887 non-null  object 
 5   End ms                          146887 non-null  float64
 6   Dur. (ms)                       146887 non-null  float64
 7   IMSI                            146887 non-null  int64  
 8   MSISDN/Number                   146887 non-null  int64  
 9   IMEI                            146887 non-null  int64  
 10  Last Location Name              146887 non-null  object 
 11  Avg RTT DL (ms)                 146887 non-null  float64
 12  Avg RTT UL (ms) 

## Metrice for user experience

In [4]:
experience_analysis_df = df[['MSISDN/Number','Avg RTT DL (ms)','Avg RTT UL (ms)','Avg Bearer TP DL (kbps)','Avg Bearer TP UL (kbps)',
    'TCP DL Retrans. Vol (Bytes)','TCP UL Retrans. Vol (Bytes)','Handset Type']]

In [9]:
experience_analysis_df['Total Avg RTT (ms)'] = experience_analysis_df['Avg RTT DL (ms)'] + experience_analysis_df['Avg RTT UL (ms)'] 
experience_analysis_df['Total Avg Bearer TP (kbps)'] = experience_analysis_df['Avg Bearer TP DL (kbps)'] + experience_analysis_df['Avg Bearer TP UL (kbps)'] 
experience_analysis_df['Total TCP Retrans. Vol (Bytes)'] = experience_analysis_df['TCP DL Retrans. Vol (Bytes)'] + experience_analysis_df['TCP UL Retrans. Vol (Bytes)']
experience_analysis_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 146887 entries, 0 to 146886
Data columns (total 11 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   MSISDN/Number                   146887 non-null  int64  
 1   Avg RTT DL (ms)                 146887 non-null  float64
 2   Avg RTT UL (ms)                 146887 non-null  float64
 3   Avg Bearer TP DL (kbps)         146887 non-null  float64
 4   Avg Bearer TP UL (kbps)         146887 non-null  float64
 5   TCP DL Retrans. Vol (Bytes)     146887 non-null  float64
 6   TCP UL Retrans. Vol (Bytes)     146887 non-null  float64
 7   Handset Type                    146887 non-null  object 
 8   Total Avg RTT (ms)              146887 non-null  float64
 9   Total Avg Bearer TP (kbps)      146887 non-null  float64
 10  Total TCP Retrans. Vol (Bytes)  146887 non-null  float64
dtypes: float64(9), int64(1), object(1)
memory usage: 12.3+ MB


C:\Users\Dell6\AppData\Local\Temp\ipykernel_20400\1774172515.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experience_analysis_df['Total Avg RTT (ms)'] = experience_analysis_df['Avg RTT DL (ms)'] + experience_analysis_df['Avg RTT UL (ms)']
C:\Users\Dell6\AppData\Local\Temp\ipykernel_20400\1774172515.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  experience_analysis_df['Total Avg Bearer TP (kbps)'] = experience_analysis_df['Avg Bearer TP DL (kbps)'] + experience_analysis_df['Avg Bearer TP UL (kbps

In [10]:
experience_analysis_df.head(5)

,MSISDN/Number,Avg RTT DL (ms),Avg RTT UL (ms),Avg Bearer TP DL (kbps),Avg Bearer TP UL (kbps),TCP DL Retrans. Vol (Bytes),TCP UL Retrans. Vol (Bytes),Handset Type,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes)
0,33664962239,42.0,5.0,23.0,44.0,19520.0,7230.0,Samsung Galaxy A5 Sm-A520F,47.0,67.0,26750.0
1,33681854413,65.0,5.0,16.0,26.0,19520.0,7230.0,Samsung Galaxy J5 (Sm-J530),70.0,42.0,26750.0
2,33760627129,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Galaxy A8 (2018),70.0,15.0,26750.0
3,33750343200,65.0,5.0,44.0,44.0,19520.0,7230.0,undefined,70.0,88.0,26750.0
4,33699795932,65.0,5.0,6.0,9.0,19520.0,7230.0,Samsung Sm-G390F,70.0,15.0,26750.0


In [13]:
experience_analysis_df = experience_analysis_df[['MSISDN/Number','Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)','Handset Type']]
experience_analysis_df.head()

,MSISDN/Number,Total Avg RTT (ms),Total Avg Bearer TP (kbps),Total TCP Retrans. Vol (Bytes),Handset Type
0,33664962239,47.0,67.0,26750.0,Samsung Galaxy A5 Sm-A520F
1,33681854413,70.0,42.0,26750.0,Samsung Galaxy J5 (Sm-J530)
2,33760627129,70.0,15.0,26750.0,Samsung Galaxy A8 (2018)
3,33750343200,70.0,88.0,26750.0,undefined
4,33699795932,70.0,15.0,26750.0,Samsung Sm-G390F


In [15]:
agg_experience_by_customer = experience_analysis_df.groupby('MSISDN/Number').agg({'Total Avg RTT (ms)': 'sum',
                                                                          'Total Avg Bearer TP (kbps)': 'sum',
                                                                          'Total TCP Retrans. Vol (Bytes)': 'sum', 
                                                                           'Handset Type': lambda x: x.mode()[0]})

In [24]:
string_columns = experience_analysis_df.select_dtypes(include='object').columns.tolist()
string_columns
convert_to_string(experience_analysis_df, string_columns)
outlier_handler = OutlierHandler(experience_analysis_df)

In [26]:
outlier_handler.getOverview(['Total Avg RTT (ms)','Total Avg Bearer TP (kbps)','Total TCP Retrans. Vol (Bytes)'])

Name of columns,Total TCP Retrans. Vol (Bytes),Total Avg RTT (ms),Total Avg Bearer TP (kbps)
Min,51.0,0.0,0.0
Q1,55408.5,37.0,90.0
Median,426677.0,52.0,126.0
Q3,3145449.5,82.0,21714.0
Max,7824924.5,150.5,52765.0
IQR,3090041.0,45.0,21624.0
Lower fence,-4579653.0,-30.5,-32346.0
Upper fence,7780511.0,149.5,54150.0
Skew,1.213194,0.940673,1.257919
Number_of_outliers,9772,3535,0
